In [1]:
import numpy as np
import time
import cv2
import math
import imutils

In [2]:
labelsPath = 'coco.names'

In [3]:
LABELS = open(labelsPath).read().strip().split("\n")

In [4]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

In [5]:
weightsPath = 'yolov3.weights'
configPath = 'yolov3.cfg'

In [6]:
print("Loading Machine Learning Model ...")

Loading Machine Learning Model ...


In [7]:
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [8]:
print("Starting Camera ...")

Starting Camera ...


In [9]:
cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret, image = cap.read()
    image = imutils.resize(image, width=800)
    (H, W) = image.shape[:2]
    ln = net.getLayerNames()
    ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()
    print("Prediction time/frame : {:.6f} seconds".format(end - start))
    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > 0.1 and classID == 0:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.3)
    ind = []
    for i in range(0,len(classIDs)):
        if(classIDs[i]==0):
            ind.append(i)
    a = []
    b = []

    if len(idxs) > 0:
            for i in idxs.flatten():
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                a.append(x)
                b.append(y)
                
    distance=[] 
    nsd = []
    for i in range(0,len(a)-1):
        for k in range(1,len(a)):
            if(k==i):
                break
            else:
                x_dist = (a[k] - a[i])
                y_dist = (b[k] - b[i])
                d = math.sqrt(x_dist * x_dist + y_dist * y_dist)
                distance.append(d)
                if(d <=220):
                    nsd.append(i)
                    nsd.append(k)
                nsd = list(dict.fromkeys(nsd))
                print(nsd)
    color = (0, 0, 255) 
    for i in nsd:
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = "Red Alert"
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
    color = (0, 255, 0) 
    if len(idxs) > 0:
        for i in idxs.flatten():
            if (i in nsd):
                break
            else:
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                text = 'Normal'
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)   
    
    cv2.imshow("Social Distancing Detector", image)
    if cv2.waitKey(2)  == 27:
        break
cap.release()
cv2.destroyAllWindows()

Prediction time/frame : 0.346077 seconds
Prediction time/frame : 0.235052 seconds
Prediction time/frame : 0.232055 seconds
Prediction time/frame : 0.222047 seconds
Prediction time/frame : 0.223047 seconds
Prediction time/frame : 0.216049 seconds
Prediction time/frame : 0.201046 seconds
Prediction time/frame : 0.197045 seconds
Prediction time/frame : 0.244052 seconds
Prediction time/frame : 0.226040 seconds
Prediction time/frame : 0.197046 seconds
Prediction time/frame : 0.202045 seconds
Prediction time/frame : 0.226049 seconds
Prediction time/frame : 0.225051 seconds
Prediction time/frame : 0.204044 seconds
Prediction time/frame : 0.212044 seconds
Prediction time/frame : 0.227051 seconds
Prediction time/frame : 0.221051 seconds
Prediction time/frame : 0.214041 seconds
Prediction time/frame : 0.202026 seconds
Prediction time/frame : 0.200038 seconds
Prediction time/frame : 0.199032 seconds
Prediction time/frame : 0.206046 seconds
Prediction time/frame : 0.196044 seconds
Prediction time/